# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [430]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [431]:
tf.__version__

'2.4.1'

### Importar o dataset

In [432]:
dataset = pd.read_csv('Churn_Modelling.csv')
#dataset = pd.read_csv('https://raw.githubusercontent.com/deeplearningunb/building-ann/master/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [433]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [434]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [435]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [436]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [437]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [438]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [439]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [440]:
ann.add(tf.keras.layers.Dense(units=5, activation='selu'))

### Adding the second hidden layer

In [441]:
ann.add(tf.keras.layers.Dense(units=4, activation='selu'))

### Adding the output layer

In [442]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [443]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [444]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.8402 - accuracy: 0.5561
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4828 - accuracy: 0.7956
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4549 - accuracy: 0.8015
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4334 - accuracy: 0.8053
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4201 - accuracy: 0.8188
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4081 - accuracy: 0.8245
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4099 - accuracy: 0.8257
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4174 - accuracy: 0.8211
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3951 - accuracy: 0.8341
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3952 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [445]:
y_pred = ann.predict(X_test)
y_pred

array([[0.22732735],
       [0.25405574],
       [0.12145758],
       ...,
       [0.16383654],
       [0.11904526],
       [0.21542141]], dtype=float32)

In [446]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [447]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1520   75]
 [ 197  208]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)